## Classical Models

In [70]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from nltk import word_tokenize     
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import jaccard_similarity_score
import numpy as np 

In [71]:
stop_words = set(stopwords.words('english'))

In [72]:
def load_data(subset = None):
    data = pd.read_csv("./dataset_20000.csv")
    if subset is not None:
        return data.head(subset)
    return data

In [73]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [74]:
dataset = load_data(subset = 5)

In [75]:
stemmer = PorterStemmer()

In [76]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [77]:
def tokenize(text):
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    filtered = [word for word in tokens if word not in stop_words]
    stems = stem_tokens(filtered, stemmer)
    return stems

In [78]:
x = dataset["summary"]
y = dataset.drop(["summary"], axis=1)

In [79]:
def sent_vectorizer(sent, model):
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model[w]
            else:
                sent_vec = np.add(sent_vec, model[w])
            numw+=1
        except:
            pass
    
    return np.asarray(sent_vec) / numw

In [80]:
mapped = x.map(tokenize)
word_model = Word2Vec(mapped, min_count=1)

In [81]:
word_model.wv.vocab

{'%': <gensim.models.keyedvectors.Vocab at 0x7f282e9139e8>,
 "''": <gensim.models.keyedvectors.Vocab at 0x7f282e96a898>,
 "'s": <gensim.models.keyedvectors.Vocab at 0x7f282e913908>,
 "'ve": <gensim.models.keyedvectors.Vocab at 0x7f2826397470>,
 ',': <gensim.models.keyedvectors.Vocab at 0x7f282e965cc0>,
 '.': <gensim.models.keyedvectors.Vocab at 0x7f2826394978>,
 '...': <gensim.models.keyedvectors.Vocab at 0x7f2826394358>,
 '1,400': <gensim.models.keyedvectors.Vocab at 0x7f282e9bcc88>,
 '1983': <gensim.models.keyedvectors.Vocab at 0x7f2826394d30>,
 '1act': <gensim.models.keyedvectors.Vocab at 0x7f28263a16d8>,
 '22nd': <gensim.models.keyedvectors.Vocab at 0x7f282e9c2da0>,
 '2act': <gensim.models.keyedvectors.Vocab at 0x7f282e9bc748>,
 '3act': <gensim.models.keyedvectors.Vocab at 0x7f282e9bcfd0>,
 '4act': <gensim.models.keyedvectors.Vocab at 0x7f2826397f28>,
 '5': <gensim.models.keyedvectors.Vocab at 0x7f282e9864e0>,
 '84': <gensim.models.keyedvectors.Vocab at 0x7f282e9bc8d0>,
 ';': <gens

In [82]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [83]:
val = word_model['proud']
print(val)

/home/ubuntu/MovieGenrePrediction/env/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


KeyError: "word 'proud' not in vocabulary"

In [87]:

word_bag = [0]*len(x)

i = 0;
for item in x: 
    word_bag[i] = list(set(tokenize(item)))
    i = i+1

word_bag = list(word_bag)
for index,item in np.ndenumerate(word_bag):
    val = word_model[item]
    word_bag[index[0]] = val
    
print(word_bag)


[array([[ 1.3801618e-03,  3.5373837e-04, -3.4910324e-03, ...,
         1.8061483e-03, -4.5948243e-03, -4.4435230e-03],
       [ 2.6681365e-03,  4.1752914e-03,  1.3033106e-04, ...,
        -1.3333979e-03,  3.7005951e-03,  2.2432229e-03],
       [ 1.5102058e-03,  3.8321060e-03,  1.9371038e-03, ...,
         7.6932600e-05,  3.6555054e-03, -3.0253356e-04],
       ...,
       [ 1.3186593e-03,  6.0923147e-04, -4.0051211e-03, ...,
         4.3993467e-03,  4.3429509e-03,  3.9775376e-03],
       [ 3.5620304e-03,  4.5506381e-03,  4.4642449e-03, ...,
         2.3981798e-04, -3.4376078e-03,  3.6829652e-03],
       [ 2.0982472e-03, -4.5725540e-03, -3.3463228e-03, ...,
         1.1188653e-03, -4.7408333e-03,  2.5123120e-03]], dtype=float32), array([[ 0.0044516 , -0.00484943, -0.00386963, ...,  0.00496512,
         0.00407195, -0.00472756],
       [-0.00273402,  0.00233608,  0.00207809, ...,  0.00421832,
         0.0027413 ,  0.00265415],
       [ 0.00415719, -0.00448456, -0.00498415, ..., -0.0034595

/home/ubuntu/MovieGenrePrediction/env/lib/python3.5/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [95]:
def build_model():
    classifier = MLPClassifier((14,))
    arr = extract_vector_array(word_bag)
    
    x_train, x_test, y_train, y_test = train_test_split(extract_vector_array(word_bag), y, test_size=0.3)
    
    print(x_train)
    model = classifier.fit(x_train, y_train)
#     predictions = model.predict(x_test)
#     target_names = dataset.drop(["summary"], axis=1).columns.tolist()
#     score = jaccard_similarity_score(y_test, predictions)
#     print(score)
#     return classification_report(y_test, predictions, target_names= target_names)

In [96]:
results = build_model()
print(results)

[[array([ 1.3801618e-03,  3.5373837e-04, -3.4910324e-03,  1.0686596e-03,
       -7.9020381e-04, -2.0472906e-03, -9.4307971e-04, -1.7329203e-03,
        1.1773156e-03, -4.8360093e-03,  8.7112997e-04, -2.2687451e-03,
        1.9945838e-03, -4.6914127e-03,  2.4736742e-03, -3.5763369e-03,
       -4.3902714e-03, -4.8550656e-03,  4.1139843e-03, -3.6911417e-03,
        3.5415071e-03, -2.7470838e-04, -4.8743230e-03,  8.4440777e-04,
        1.6228370e-04, -5.2881299e-04,  1.0570295e-03, -3.5708812e-03,
        3.3927704e-03, -1.8183906e-04,  2.9776995e-03, -3.4008885e-04,
        6.0396106e-04, -2.0909915e-03,  4.3742307e-03,  2.4610902e-03,
        4.2659007e-03,  1.5661951e-04,  4.7690906e-03,  3.4050697e-03,
       -2.3680109e-04, -2.6023118e-03,  4.9472065e-04,  1.2776614e-03,
       -3.3252577e-03, -4.4368943e-03,  3.8708446e-03, -2.0295895e-04,
        4.2233467e-03,  8.7930617e-04,  2.1575955e-03,  1.1579349e-03,
       -3.3977828e-03,  3.4239241e-03, -3.1138749e-03,  1.6344052e-03,
    

ValueError: Expected 2D array, got 1D array instead:
array=[list([array([ 1.3801618e-03,  3.5373837e-04, -3.4910324e-03,  1.0686596e-03,
       -7.9020381e-04, -2.0472906e-03, -9.4307971e-04, -1.7329203e-03,
        1.1773156e-03, -4.8360093e-03,  8.7112997e-04, -2.2687451e-03,
        1.9945838e-03, -4.6914127e-03,  2.4736742e-03, -3.5763369e-03,
       -4.3902714e-03, -4.8550656e-03,  4.1139843e-03, -3.6911417e-03,
        3.5415071e-03, -2.7470838e-04, -4.8743230e-03,  8.4440777e-04,
        1.6228370e-04, -5.2881299e-04,  1.0570295e-03, -3.5708812e-03,
        3.3927704e-03, -1.8183906e-04,  2.9776995e-03, -3.4008885e-04,
        6.0396106e-04, -2.0909915e-03,  4.3742307e-03,  2.4610902e-03,
        4.2659007e-03,  1.5661951e-04,  4.7690906e-03,  3.4050697e-03,
       -2.3680109e-04, -2.6023118e-03,  4.9472065e-04,  1.2776614e-03,
       -3.3252577e-03, -4.4368943e-03,  3.8708446e-03, -2.0295895e-04,
        4.2233467e-03,  8.7930617e-04,  2.1575955e-03,  1.1579349e-03,
       -3.3

In [88]:
def extract_vector_array(test):
    arr = []
    
    for i, vector in enumerate(test):
        arr.append([])
        for j, elem in enumerate(vector):
            arr[i].append(elem)
            
    return arr